In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch, os, re, pandas as pd, json
import torch
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

import pandas as pd
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch import optim
import gc
from torch.nn import DataParallel
import nltk
import random
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
device = "cuda:7" if torch.cuda.is_available() else "cpu"

In [4]:
print(str(device))

cuda:7


In [5]:
!pwd

/home/ubuntu


In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [7]:
head_model = GPT2LMHeadModel.from_pretrained('gpt2')

In [8]:
pos_model_dict = torch.load('/home/ubuntu/ImdbPOS_model/pytorch_model.bin', map_location='cuda')
neg_model_dict = torch.load('/home/ubuntu/ImdbNeg_model/pytorch_model.bin', map_location='cuda')
gatsby_model_dict = torch.load('/home/ubuntu/FTGatsby_model/pytorch_model.bin', map_location='cuda')
origin_model_dict = torch.load('/home/ubuntu/FTOrigin_model/pytorch_model.bin', map_location='cuda') 
#ImdbPOS_model/pytorch_model.bin
#/home/ubuntu/ImdbPOS_model
#ImdbNeg_model

In [9]:
head_model_pos = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=pos_model_dict, pad_token_id=tokenizer.eos_token_id).to(device)
head_model_neg = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=neg_model_dict, pad_token_id=tokenizer.eos_token_id).to(device)

head_model_gatsby = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=gatsby_model_dict, pad_token_id=tokenizer.eos_token_id).to(device)
head_model_origin = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=origin_model_dict, pad_token_id=tokenizer.eos_token_id).to(device)

base_gpt2_head = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id).to(device)

In [10]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [11]:
gatsby_sen_list = nltk.sent_tokenize(gatsby)
origin_sen_list = nltk.sent_tokenize(origin)
print(len(gatsby_sen_list))
print(len(origin_sen_list))

2439
3981


In [70]:
# ref https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
import re
import string
from collections import Counter
from collections import defaultdict
gatsby_first_words_list = []
for sen_g in gatsby_sen_list:
    sen_g = re.sub(r'^\s*|\s\s*', ' ', sen_g).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_g = filter(lambda x: x in printable, sen_g)
    sen_g = ''.join(filter(lambda x: x in printable, sen_g))
    sen_g_list = sen_g.split()
    #print(sen_g_list[0:3])
    gatsby_first_words_list.append(sen_g_list[0:3])
    
origin_first_words_list = []  
for sen_o in origin_sen_list:
    sen_o = re.sub(r'^\s*|\s\s*', ' ', sen_o).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_o = filter(lambda x: x in printable, sen_o)
    sen_o = ''.join(filter(lambda x: x in printable, sen_o))
    sen_o_list = sen_o.split()
    # print(sen_o_list[0:3])
    origin_first_words_list.append(sen_o_list[0:3]) 
    
initial_unigrams_gatsby = defaultdict(int)
initial_bigrams_gatsby = defaultdict(int)
initial_trigrams_gatsby = defaultdict(int)
initial_unigrams_origin = defaultdict(int)
initial_bigrams_origin = defaultdict(int)
initial_trigrams_origin = defaultdict(int)
for first_words in gatsby_first_words_list:
    initial_unigrams_gatsby[first_words[0]] += 1
    initial_bigrams_gatsby[' '.join(first_words[:2])] += 1
    initial_trigrams_gatsby[' '.join(first_words)] += 1
    
for first_words in origin_first_words_list:
    initial_unigrams_origin[first_words[0]] += 1
    initial_bigrams_origin[' '.join(first_words[:2])] += 1
    initial_trigrams_origin[' '.join(first_words)] += 1
    
unigram_overlap = set(initial_unigrams_gatsby.keys())
unigram_overlap = unigram_overlap.intersection(initial_unigrams_origin.keys())

bigram_overlap = set(initial_bigrams_gatsby.keys())
bigram_overlap = bigram_overlap.intersection(initial_bigrams_origin.keys())
   
trigram_overlap = set(initial_trigrams_gatsby.keys())
trigram_overlap = trigram_overlap.intersection(initial_trigrams_origin.keys())


# print(f"unigram_overlap: {str(unigram_overlap)}")
# print("\n")
print(f"bigram_overlap: {str(bigram_overlap)}")
print("\n")
# print(f"trigram_overlap: {str(trigram_overlap)}")

bigram_list = list(bigram_overlap)
    
# combine the list

# combined_g_o_list = gatsby_first_words_list + origin_first_words_list

# mini_sen_list = []
# for begin in combined_g_o_list:
#     mini_sen_list.append(' '.join(begin))
    
# # ref https://stackoverflow.com/questions/43473736/most-common-2-grams-using-python
# bigrams = zip(mini_sen_list, mini_sen_list[1:])
# counts = Counter(bigrams)
# print(counts.most_common())

bigram_overlap: {'The fact', 'And what', 'But no', 'As we', 'The most', 'Do they', 'But when', 'But we', 'Some little', 'But the', 'One of', 'He always', 'And now', 'I see', 'It occurred', 'As soon', 'No one', 'The flowers', 'There is', 'In the', 'In fact,', 'There must', 'I cannot', 'Even when', 'The practical', 'I should', 'In this', 'At the', 'Now it', 'The day', 'As he', 'It was', 'Most of', 'The only', 'At first', 'At a', 'I could', 'Nothing at', 'We could', 'The other', 'I remember', 'The old', 'A few', 'But what', 'In one', 'Now I', 'If we', 'So we', 'The masters', 'Let the', 'All the', 'How the', 'By the', 'But how', 'At this', 'See how', 'On a', 'But it', 'I will', 'When we', 'After the', 'In my', 'And when', 'How much', 'I must', 'I would', 'It makes', 'Not only', 'If there', 'And the', 'But I', 'He would', 'A large', 'To my', 'I believe', 'This was', 'This is', 'The eyes', 'It seems', 'A little', 'And I', 'If it', 'With the', 'I know', 'For several', 'As I', 'Now, in', 'Afte

In [13]:
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [14]:
# head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)
# # head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
# head_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
# head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
standard_gpt = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

In [15]:
# model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
# head_model.load_state_dict(torch.load("/home/ubuntu/pos_imdb_model"))
# head_model.load_state_dict(torch.load("/home/ubuntu/small_neg_midterm_version"))
# head_model.eval()


In [16]:
# type(head_model)

In [17]:
!pwd

/home/ubuntu


In [25]:
import pandas as pd
  
# list of strings
# lst = ['I', 'Movie', 'He', 'What', 
#             'The', 'We', 'I thought', 'I sat', 'How about', 'It was', "This is going", "The nature", ]
  
lst = random.sample(bigram_list,100)
# Calling DataFrame constructor on list
df = pd.DataFrame(lst)
df

,0
0,Nothing at
1,The only
2,When we
3,After a
4,On the
...,...
95,As he
96,I was
97,From the
98,At a


In [28]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=100,
    entry_length=50, #maximum number of words
    top_p=0.5,
    temperature=1,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated.to(device), labels=generated.to(device))
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated.to(device), next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().cpu().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().cpu().numpy())
                # output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                output_text = f"{tokenizer.decode(output_list)}" 
                generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data, model, tokenizer):
    generated_lyrics = []
    for i in range(len(test_data)):
        x = generate(model, tokenizer, test_data[0][i], entry_count=1)
        generated_lyrics.append(x)
    return generated_lyrics

#Run the functions to generate the lyrics
# generated_text = text_generation(df, head_model, base_tokenizer)

In [29]:
generated_text_pos = text_generation(df, head_model_pos, tokenizer)
for text in generated_text_pos:
    print(text, "\n")

100%|██████████| 1/1 [00:00<00:00,  2.69it/s]

['Nothing at all. It is very funny and a bit too good to be true.\nA few years ago, when I was growing up, I was fortunate enough to be able to watch a movie that was based on the novels of William S. Burrough'] 

['The only thing I would change is to go back to the time of "War on Terror." The New York City Police Department\'s New York Field Office was at the time a law enforcement agency. It was the first time that they ever had a Field Office.'] 

["When we go to a mall or on a subway, we often find ourselves looking at the billboards and wondering what they are about. We are reminded of the beautiful way the city has always been, but it's been replaced by a digitalized world that has become"] 

['After a series of missteps, and a hard time keeping up with the latest, there is a lot to say about the film. The cast is as good as ever, with Antonioni as the chameleon, as the nymph, as the'] 

["On the other hand, when a big corporation can't pay for its own corporate welfare, they ca

In [30]:
generated_text_neg = text_generation(df, head_model_neg, tokenizer)
for text in generated_text_neg:
    print(text, "\n")

100%|██████████| 1/1 [00:00<00:00,  2.73it/s]

["Nothing at all.I was in love with the series and wanted to watch it again, but it just didn't work out. It's just too bad that I didn't get to see the original version of it. I just didn't get to see it"] 

["The only reason I could give for this film is because it's so much fun to watch it. It's so hard to believe that I was able to watch it without any clue what it was about. I was confused, and I felt a little bit disappointed"] 

['When we are told that it is in the movie, we are disappointed. What is this movie? It is a real disappointment to me. It is not about me, it is about me. I was waiting for a bad movie to happen, but this movie'] 

["After a while, you can get the movie going, but it doesn't really make sense. I didn't even get the sense that it was going to be about a couple of times, and it didn't really make sense to me at all. I thought"] 

["On the surface, the character's story is about an English man who is obsessed with being famous. He's not, and he's not a ver

In [31]:
generated_text_gatsby = text_generation(df, head_model_gatsby, tokenizer)
for text in generated_text_gatsby:
    print(text, "\n")


100%|██████████| 1/1 [00:00<00:00,  2.69it/s]

['Nothing at all. She had been a girl for years. She was born to a wealthy family in Wisconsin, and was the daughter of a wealthy husband and two beautiful young daughters. She was married to a wealthy and prosperous man named Gatsby. She was'] 

["The only thing I know about her is that she was once a spy. Her father, and I think that I know her too. She was an officer, she said.\nI don't know what he was, she said. I have no idea."] 

["When we're looking at the photographs of the last couple of weeks, I have to say, it's not the most pleasant time of my life. I've had a lot of other people around me who are just looking at me. I'm afraid to say"] 

['After a few moments, I felt that I had finally found my body, and it was hard to believe that my arm had just fallen into the car.\nWhen I got to the garage, the car was parked next to a grassy hill, and I'] 

['On the morning of August 6, 1922, an army of sixteen hundred thousand men from all parts of the country and fifty thousand fro

In [32]:
generated_text_origin = text_generation(df, head_model_origin, tokenizer)
for text in generated_text_origin:
    print(text, "\n")

100%|██████████| 1/1 [00:00<00:00,  2.71it/s]

['Nothing at all, but I can see why a single species should be so highly valued by all. But I do not doubt that it is the same with plants, for I am well aware that a plant is sometimes valued more than another for its growth; but'] 

['The only one of these cases which I can recall is that of the double stamens, the other being quite unknown. In a quarter of the country, as in some parts of the United States, there is a great scarcity of cattle and sheep; and'] 

['When we have not even begun to discuss the effects of climate on man, it will be necessary to recapitulate a few remarks on the relation of man to nature. The subject has been discussed in a very limited manner in the work on natural history. It'] 

['After a long and slow course of action, the forms of life will adapt to their new conditions, and will produce new forms of life. This is the principle which has enabled us to see that the laws of life, as well as those of the economy of'] 

['On the left side of the right sid

In [33]:
generated_text_standard = text_generation(df, standard_gpt, tokenizer)
for text in generated_text_standard:
    print(text, "\n")

100%|██████████| 1/1 [00:00<00:00,  2.72it/s]

["Nothing at all, as you can see.\n\nWhat we're seeing is a double standard.\n\nPeople are saying that because you have a license, you can't drive, but you can't have sex. They're saying that because you have a"] 

['The only reason I\'d want to write a book is because I think it\'s very exciting.\n\nYou know, you can say that the last time I wrote a book, I had a lot of friends who\'d read it and were like, "Oh'] 

['When we talk about vaccines, there\'s this notion that if you have an infection, you should have a vaccine," says Dr. Paul Griggs, a microbiologist at Harvard Medical School. "And if you have a vaccine, you should have a vaccine that'] 

['After a few minutes of quiet deliberation, the judge issued a temporary restraining order.\n\nIn the video, Officer Rosario can be heard telling the officers to get out of the car, but they can\'t.\n\n"You\'re a cop,'] 

['On the other hand, it is important to note that the most likely explanation for the presence of these negative ster

In [61]:

def clean_strings(strings):
    cleaned_strings = []
    #print(strings[:20])
    for sen in strings:
        #print(sen)
        sen = re.sub(r"\\", "", sen[0])    # bizarrely, each string is inside of a list containing nothing but that string
        sen = re.sub(r"\n", " ", sen)
        sen = filter(lambda x: x in printable, sen)
        sen = ''.join(filter(lambda x: x in printable, sen))
        cleaned_strings.append(sen)
    #print(cleaned_strings[:20])
    
    return cleaned_strings

cleaned_generated_text_neg = clean_strings(generated_text_neg)
cleaned_generated_text_pos = clean_strings(generated_text_pos)
cleaned_generated_text_gatsby = clean_strings(generated_text_gatsby)
cleaned_generated_text_origin = clean_strings(generated_text_origin)
cleaned_generated_text_standard = clean_strings(generated_text_standard)

#for sen_o in origin_shortened:
#    sen_o = re.sub(r'^\s*|\s\s*', ' ', sen_o).strip().rstrip().lstrip()
#    printable = set(string.printable)
#    sen_o = filter(lambda x: x in printable, sen_o)
#    sen_o = ''.join(filter(lambda x: x in printable, sen_o))
#    origin_short_clean.append(sen_o)
    

In [66]:
all_seqs = []
for seq in cleaned_generated_text_neg:
    all_seqs.append(('FINETUNED_NEG', seq))
for seq in cleaned_generated_text_pos:
    all_seqs.append(('FINETUNED_POS', seq))
for seq in cleaned_generated_text_gatsby:
    all_seqs.append(('FINETUNED_GATSBY', seq))
for seq in cleaned_generated_text_origin:
    all_seqs.append(('FINETUNED_ORIGIN', seq))
for seq in cleaned_generated_text_standard:
    all_seqs.append(('GPT_STANDARD', seq))
    


In [67]:
print(all_seqs)

[('FINETUNED_NEG', "Nothing at all.I was in love with the series and wanted to watch it again, but it just didn't work out. It's just too bad that I didn't get to see the original version of it. I just didn't get to see it"), ('FINETUNED_NEG', "The only reason I could give for this film is because it's so much fun to watch it. It's so hard to believe that I was able to watch it without any clue what it was about. I was confused, and I felt a little bit disappointed"), ('FINETUNED_NEG', 'When we are told that it is in the movie, we are disappointed. What is this movie? It is a real disappointment to me. It is not about me, it is about me. I was waiting for a bad movie to happen, but this movie'), ('FINETUNED_NEG', "After a while, you can get the movie going, but it doesn't really make sense. I didn't even get the sense that it was going to be about a couple of times, and it didn't really make sense to me at all. I thought"), ('FINETUNED_NEG', "On the surface, the character's story is ab

In [69]:
df = pd.DataFrame(all_seqs, columns = ['Source', 'Sequence'])

df.to_csv('finetuened.csv', encoding='utf-8', index = False)